In [ ]:
!pip install ccxt pandas tdqm numpy plotly python-dotenv nest-asyncio kaleido


In [ ]:
import asyncio
import ccxt.async_support as ccxt
import pandas as pd
import numpy as np
from datetime import datetime 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from dotenv import load_dotenv
import nest_asyncio
from tqdm import tqdm
import multiprocessing as mp
from concurrent.futures import ThreadPoolExecutor
from functools import partial

#V1
nest_asyncio.apply()

class TradingScanner:
    def __init__(self):
        load_dotenv()
        self.api_key = os.getenv('BINANCE_API_KEY')
        self.api_secret = os.getenv('BINANCE_API_SECRET')
        self.init_exchange_config()
        
        self.timeframes = ['15m', '1h', '4h', '1d']
        self.min_volume = 1_000_000
        
        self.timeframe_names = {
            '15m': '15 Minutes',
            '1h': '1 Hour',
            '4h': '4 Hours',
            '1d': '1 Day'
        }
        
        self.timeframe_limits = {
            '15m': 200,
            '1h': 150,
            '4h': 150,
            '1d': 150
        }
        
        self.lookback_periods = {
            '15m': 96,
            '1h': 72,
            '4h': 60,
            '1d': 60
        }
        
        self.excluded_coins = {
            'USDC', 'USDT', 'USTC', 'FDUSD', 'BUSD', 
            'TUSD', 'DAI', 'UST', 'USDD', 'USDP', 'SUSD'
        }

    def init_exchange_config(self):
        """Initialize exchange configuration"""
        self.exchange_config = {
            'apiKey': self.api_key,
            'secret': self.api_secret,
            'enableRateLimit': True,
            'timeout': 30000,
            'options': {
                'defaultType': 'spot',
                'adjustForTimeDifference': True,
                'recvWindow': 60000
            }
        }

    def create_exchange(self):
        """Create a new exchange instance for each process"""
        return ccxt.binance(self.exchange_config)

    def calculate_squeeze(self, ohlcv, period=20):
        """Calculate TTM Squeeze"""
        if len(ohlcv) < period:
            return False
            
        close = ohlcv[:, 4]
        high = ohlcv[:, 2]
        low = ohlcv[:, 3]
        
        middle = np.mean(close[-period:])
        stddev = np.std(close[-period:])
        bb_upper = middle + (2 * stddev)
        bb_lower = middle - (2 * stddev)
        
        tr = np.maximum.reduce([
            high[-period:] - low[-period:],
            np.abs(high[-period:] - np.roll(close[-period:], 1)),
            np.abs(low[-period:] - np.roll(close[-period:], 1))
        ])
        atr = np.mean(tr)
        kc_upper = middle + (1.5 * atr)
        kc_lower = middle - (1.5 * atr)
        
        return (bb_lower > kc_lower) and (bb_upper < kc_upper)

    def calculate_fibonacci_gradient(self, df, timeframe):
        """Calculate Fibonacci gradient"""
        lookback = self.lookback_periods[timeframe]
        gradients = np.zeros(len(df))
        
        for i in range(lookback, len(df)):
            window = df.iloc[i-lookback:i]
            high = window['high'].astype(float)
            low = window['low'].astype(float)
            close = float(df['close'].iloc[i])
            
            fib_max = high.max()
            fib_min = low.min()
            max_min_diff = fib_max - fib_min
            
            if max_min_diff <= 0:
                continue
                
            fib_sell = fib_max - (max_min_diff * 0.382)
            fib_buy = fib_max - (max_min_diff * 0.707)
            
            if close >= fib_sell:
                sell_diff = ((close - fib_sell) / max_min_diff * 100)
                gradient = ((sell_diff * 2617.801) + 100000) / 2
            elif close <= fib_buy:
                buy_diff = ((fib_buy - close) / max_min_diff * 100)
                gradient = ((buy_diff * -2617.801) + 100000) / 2
            else:
                gradient = 50000
                
            gradients[i] = (100 - gradient/1000) * -1
            
        return gradients

    async def get_active_markets(self):
        """Scan and filter active markets"""
        exchange = self.create_exchange()
        try:
            print("\nScanning active markets...")
            markets = await exchange.load_markets()
            active_pairs = []
            
            usdt_pairs = [s for s in markets if s.endswith('/USDT')]
            for symbol in tqdm(usdt_pairs, desc="Market Taraması", ncols=100):  # İlk 50 market ile test
                try:
                    ticker = await exchange.fetch_ticker(symbol)
                    if ticker['quoteVolume'] >= self.min_volume:
                        active_pairs.append(symbol)
                except Exception as e:
                    continue
                    
                await asyncio.sleep(0.5)  # Rate limit için bekleme süresi arttırıldı
            
            return active_pairs
        finally:
            await exchange.close()

    async def process_market(self, symbol):
        """Analyze a single market"""
        exchange = self.create_exchange()
        try:
            results, ohlcv_data = await self.analyze_market(exchange, symbol)
            if results and symbol in results:
                return symbol, results[symbol], ohlcv_data
            return None
        finally:
            await exchange.close()

    async def analyze_market(self, exchange, symbol):
        """Market analysis function"""
        results = {}
        ohlcv_data = {}
        timeframe_data = {}
        
        for timeframe in self.timeframes:
            try:
                ohlcv = await exchange.fetch_ohlcv(
                    symbol, 
                    timeframe, 
                    limit=self.timeframe_limits[timeframe]
                )
                
                if not ohlcv or len(ohlcv) < self.lookback_periods[timeframe]:
                    print(f"Insufficient data: {symbol} - {timeframe}")
                    return results, ohlcv_data
                    
                timeframe_data[timeframe] = {
                    'ohlcv': ohlcv,
                    'array': np.array(ohlcv),
                    'df': pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
                }
                
            except Exception as e:
                print(f"Data fetch error ({symbol} - {timeframe}): {str(e)}")
                return results, ohlcv_data
                
            await asyncio.sleep(0.5)

        all_signals = {}
        for timeframe, data in timeframe_data.items():
            gradients = self.calculate_fibonacci_gradient(data['df'], timeframe)
            last_gradient = gradients[-1]
            
            if last_gradient <= -70.7:
                signal = "Buy"
            elif last_gradient >= -38.2:
                signal = "Sell"
            else:
                return results, ohlcv_data
                
            all_signals[timeframe] = {
                'signal': signal,
                'gradient': last_gradient
            }
        
        first_signal = list(all_signals.values())[0]['signal']
        if not all(signal['signal'] == first_signal for signal in all_signals.values()):
            return results, ohlcv_data
            
        has_squeeze = False
        for timeframe, data in timeframe_data.items():
            if self.calculate_squeeze(data['array']):
                has_squeeze = True
                break
        
        if has_squeeze:
            results[symbol] = all_signals
            ohlcv_data = {tf: data['ohlcv'] for tf, data in timeframe_data.items()}
            
        return results, ohlcv_data

    def plot_analysis(self, symbol, data, ohlcv_data):
        """Chart creation function"""
        fig = make_subplots(
            rows=2, 
            cols=2, 
            subplot_titles=[f"{symbol} - {self.timeframe_names[tf]}" for tf in self.timeframes],
            vertical_spacing=0.1,
            horizontal_spacing=0.05
        )

        colors = {
            '15m': 'rgba(255, 0, 0, 0.3)', 
            '1h': 'rgba(0, 255, 0, 0.3)', 
            '4h': 'rgba(0, 0, 255, 0.3)',
            '1d': 'rgba(255, 165, 0, 0.3)'
        }

        for idx, timeframe in enumerate(self.timeframes):
            row = idx // 2 + 1
            col = idx % 2 + 1
            
            if timeframe in ohlcv_data:
                ohlcv = ohlcv_data[timeframe]
                df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
                df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
                
                gradients = self.calculate_fibonacci_gradient(df, timeframe)
                df['gradient'] = gradients
                
                fig.add_trace(
                    go.Candlestick(
                        x=df['timestamp'],
                        open=df['open'],
                        high=df['high'],
                        low=df['low'],
                        close=df['close'],
                        name=timeframe,
                        showlegend=False,
                        customdata=df['gradient'],
                        hovertext=[
                            f'Date: {date}<br>' +
                            f'Open: {open:.8f}<br>' +
                            f'High: {high:.8f}<br>' +
                            f'Low: {low:.8f}<br>' +
                            f'Close: {close:.8f}<br>' +
                            f'Gradient: {gradient:.2f}'
                            for date, open, high, low, close, gradient in zip(
                                df['timestamp'], df['open'], df['high'], 
                                df['low'], df['close'], df['gradient']
                            )
                        ],
                        hoverinfo='text'
                    ),
                    row=row, col=col
                )

                for i in range(1, len(df)):
                    is_squeeze = self.calculate_squeeze(np.array(ohlcv[max(0, i-20):i+1]))
                    if is_squeeze:
                        fig.add_shape(
                            type="rect",
                            x0=df['timestamp'].iloc[i-1],
                            x1=df['timestamp'].iloc[i],
                            y0=df['low'].min(),
                            y1=df['high'].max(),
                            fillcolor=colors[timeframe],
                            opacity=0.3,
                            layer="below",
                            line_width=0,
                            row=row,
                            col=col
                        )

                if timeframe in data.get(symbol, {}):
                    signal_info = data[symbol][timeframe]
                    if signal_info['signal'] == "Buy":
                        fig.add_trace(
                            go.Scatter(
                                x=[df['timestamp'].iloc[-1]],
                                y=[df['low'].iloc[-1] * 0.999],
                                mode='markers',
                                marker=dict(symbol='triangle-up', size=12, color='lime'),
                                name='Buy Signal',
                                showlegend=idx == 0
                            ),
                            row=row, col=col
                        )
                    elif signal_info['signal'] == "Sell":
                        fig.add_trace(
                            go.Scatter(
                                x=[df['timestamp'].iloc[-1]],
                                y=[df['high'].iloc[-1] * 1.001],
                                mode='markers',
                                marker=dict(symbol='triangle-down', size=12, color='red'),
                                name='Sell Signal',
                                showlegend=idx == 0
                            ),
                            row=row, col=col
                        )

                fig.update_xaxes(
                    title_text="Date",
                    gridcolor='rgba(128,128,128,0.2)',
                    rangeslider=dict(visible=False),
                    row=row, 
                    col=col,
                    tickfont=dict(size=10)
                )
                fig.update_yaxes(
                    title_text="Price",
                    gridcolor='rgba(128,128,128,0.2)',
                    row=row, 
                    col=col,
                    tickformat='.8f',
                    tickfont=dict(size=10)
                )

        # General chart style
        fig.update_layout(
            height=1200,
            width=1800,
            template="plotly_dark",
            paper_bgcolor='black',
            plot_bgcolor='black',
            title=dict(
                text=f"{symbol}",
                font=dict(size=28, color='white'),
                x=0.5,
                y=0.95
            ),
            showlegend=True,
            legend=dict(
                yanchor="top",
                y=0.99,
                xanchor="right",
                x=0.99,
                bgcolor='rgba(0,0,0,0.5)',
                font=dict(size=12, color='white')
            ),
            margin=dict(t=80, b=50, l=50, r=50)
        )

        # Color adjustment for subplot titles
        for annotation in fig['layout']['annotations']:
            annotation['font'] = dict(size=14, color='white')

        return fig

    async def run_analysis(self):
        """Main analysis function - optimized with multiprocessing"""
        try:
            # Get active markets
            markets = await self.get_active_markets()
            print(f"\nTotal {len(markets)} active markets found")
            print(f"Markets to be processed: {', '.join(markets)}\n")

            # Get the number of CPU cores (reserve 1 core)
            num_processes = max(1, mp.cpu_count() - 1)
            
            # Split markets according to the number of processes
            market_chunks = self.chunk_markets(markets, num_processes)
            
            # Parallel processing with ProcessPoolExecutor
            with concurrent.futures.ProcessPoolExecutor(max_workers=num_processes) as executor:
                # Start a separate process for each chunk
                loop = asyncio.get_event_loop()
                futures = [
                    loop.run_in_executor(executor, 
                        lambda x: asyncio.run(self.run_parallel_analysis(x)), 
                        chunk)
                    for chunk in market_chunks
                ]
                
                # Wait for all processes to complete
                results = []
                for future in tqdm(
                    concurrent.futures.as_completed(futures),
                    total=len(futures),
                    desc="Analiz İlerlemesi",
                    ncols=100
                ):
                    chunk_results = await future
                    results.extend([r for r in chunk_results if r is not None])

            # Process the results
            print("\n=== SUMMARY ===")
            print(f"Total signals found for {len(results)} coins:")
            
            for symbol, signals, ohlcv_data in results:
                print(f"\n{symbol} Signal Summary:")
                for tf, result in signals.items():
                    print(f"✓ {self.timeframe_names[tf]}: {result['signal']} ({result['gradient']:.2f})")
                
                # Create chart
                fig = self.plot_analysis(symbol, {symbol: signals}, ohlcv_data)
                filename = f"{symbol.replace('/', '_')}_{datetime.now().strftime('%Y%m%d_%H%M')}.png"
                fig.write_image(filename, engine="kaleido", scale=2)

        except Exception as e:
            print(f"General error: {str(e)}")
            raise e

    async def process_chunk(self, chunk):
        """Process a group of markets"""
        tasks = []
        for symbol in chunk:
            tasks.append(self.process_market(symbol))
            await asyncio.sleep(0.1)  # Rate limit kontrolü
        return await asyncio.gather(*tasks, return_exceptions=True)

    async def run_analysis(self):
        """Main analysis function"""
        try:
            markets = await self.get_active_markets()
            if not markets:
                print("No active markets found!")
                return

            print(f"\nTotal {len(markets)} active markets found")
            
            # Use smaller chunks
            chunk_size = 5  # Chunk size reduced
            chunks = [markets[i:i + chunk_size] for i in range(0, len(markets), chunk_size)]
            
            results = []
            for chunk in tqdm(chunks, desc="Analiz İlerlemesi", ncols=100):
                try:
                    chunk_results = await self.process_chunk(chunk)
                    valid_results = [r for r in chunk_results if r is not None and not isinstance(r, Exception)]
                    results.extend(valid_results)
                    await asyncio.sleep(1)  # Chunklar arası bekleme
                except Exception as e:
                    print(f"Chunk processing error: {str(e)}")
                    continue

            print("\n=== SUMMARY ===")
            print(f"Total signals found for {len(results)} coins:")
            
            for symbol, signals, ohlcv_data in results:
                try:
                    print(f"\n{symbol} Signal Summary:")
                    for tf, result in signals.items():
                        print(f"✓ {self.timeframe_names[tf]}: {result['signal']} ({result['gradient']:.2f})")
                    
                    fig = self.plot_analysis(symbol, {symbol: signals}, ohlcv_data)
                    filename = f"{symbol.replace('/', '_')}_{datetime.now().strftime('%Y%m%d_%H%M')}.png"
                    fig.write_image(filename, engine="kaleido", scale=2)
                    
                    # Clear memory
                    del fig
                except Exception as e:
                    print(f"Graph creation error for {symbol}: {str(e)}")
                    continue

        except Exception as e:
            print(f"General error: {str(e)}")
            import traceback
            traceback.print_exc()

    @classmethod
    async def create_and_run(cls):
        """Factory method to create and run an instance of the class"""
        scanner = cls()
        await scanner.run_analysis()
        return scanner

def run_scanner():
    """Run function for Jupyter"""
    try:
        scanner = TradingScanner()
        asyncio.run(scanner.run_analysis())
    except Exception as e:
        print(f"\nAn error occurred: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    run_scanner()



Aktif marketler taranıyor...


Market Taraması: 100%|████████████████████████████████████████████| 555/555 [06:56<00:00,  1.33it/s]



Toplam 371 aktif market bulundu


Analiz İlerlemesi:  95%|█████████████████████████████████████████▋  | 71/75 [12:21<00:41, 10.50s/it]

Yetersiz veri: COOKIE/USDT - 1d
Yetersiz veri: S/USDT - 1d
Yetersiz veri: D/USDT - 1d
Yetersiz veri: CGPT/USDT - 1d
Yetersiz veri: AIXBT/USDT - 1d


...

Yetersiz veri: SHELL/USDT - 4h
Yetersiz veri: KAITO/USDT - 1d
Yetersiz veri: TST/USDT - 1d
Yetersiz veri: LAYER/USDT - 1d
Yetersiz veri: HEI/USDT - 1d


Analiz İlerlemesi:  99%|███████████████████████████████████████████▍| 74/75 [12:51<00:10, 10.06s/it]

Yetersiz veri: RED/USDT - 4h


Analiz İlerlemesi: 100%|████████████████████████████████████████████| 75/75 [12:56<00:00, 10.36s/it]



=== ÖZET ===
Toplam 8 coin için sinyal bulundu:

TFUEL/USDT Sinyal Özeti:
✓ 15 Dakika: Alış (-74.60)
✓ 1 Saat: Alış (-74.60)
✓ 4 Saat: Alış (-73.42)
✓ 1 Gün: Alış (-83.23)

RARE/USDT Sinyal Özeti:
✓ 15 Dakika: Alış (-84.13)
✓ 1 Saat: Alış (-84.13)
✓ 4 Saat: Alış (-84.80)
✓ 1 Gün: Alış (-84.61)

ASTR/USDT Sinyal Özeti:
✓ 15 Dakika: Alış (-85.16)
✓ 1 Saat: Alış (-85.16)
✓ 4 Saat: Alış (-77.64)
✓ 1 Gün: Alış (-70.78)

RPL/USDT Sinyal Özeti:
✓ 15 Dakika: Alış (-80.47)
✓ 1 Saat: Alış (-80.47)
✓ 4 Saat: Alış (-74.78)
✓ 1 Gün: Alış (-71.49)

BANANA/USDT Sinyal Özeti:
✓ 15 Dakika: Alış (-86.09)
✓ 1 Saat: Alış (-78.77)
✓ 4 Saat: Alış (-78.82)
✓ 1 Gün: Alış (-85.98)

EURI/USDT Sinyal Özeti:
✓ 15 Dakika: Satış (-4.03)
✓ 1 Saat: Satış (-4.03)
✓ 4 Saat: Satış (-20.76)
✓ 1 Gün: Satış (-13.70)

SCR/USDT Sinyal Özeti:
✓ 15 Dakika: Alış (-84.99)
✓ 1 Saat: Alış (-93.73)
✓ 4 Saat: Alış (-81.28)
✓ 1 Gün: Alış (-72.33)

COW/USDT Sinyal Özeti:
✓ 15 Dakika: Alış (-85.72)
✓ 1 Saat: Alış (-89.09)
✓ 4 Saat: Al

In [ ]:
import asyncio
import ccxt.async_support as ccxt
import pandas as pd
import numpy as np
from datetime import datetime 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from dotenv import load_dotenv
import nest_asyncio
from tqdm import tqdm
import multiprocessing as mp
from concurrent.futures import ThreadPoolExecutor
from functools import partial
from scipy.signal import argrelextrema

#V2
nest_asyncio.apply()

class TradingScanner:
    def __init__(self):
        load_dotenv()
        self.api_key = os.getenv('BINANCE_API_KEY')
        self.api_secret = os.getenv('BINANCE_API_SECRET')
        self.init_exchange_config()
        
        self.timeframes = ['15m', '1h', '4h', '1d']
        self.min_volume = 1_000_000
        
        self.timeframe_names = {
            '15m': '15 Dakika',
            '1h': '1 Saat',
            '4h': '4 Saat',
            '1d': '1 Gün'
        }
        
        self.timeframe_limits = {
            '15m': 200,
            '1h': 150,
            '4h': 150,
            '1d': 150
        }
        
        self.lookback_periods = {
            '15m': 96,
            '1h': 72,
            '4h': 60,
            '1d': 60
        }
        
        # Destek/direnç seviyelerinin tespit edilmesi için parametreler
        self.sr_order = 5  # Yerel minimum/maksimumları bulmak için bakılacak nokta sayısı
        self.sr_threshold = 0.02  # Fiyatta minimum % fark (birbirine çok yakın seviyeleri eleme)
        self.sr_lookback = 100  # Son kaç mum üzerinden destek/direnç analizi yapılacak
        self.sr_touch_count = 2  # Minimum kaç kez test edilmiş seviyeler geçerli sayılacak
        
        # Piyasa korelasyonu için ana coin
        self.market_leader = 'BTC/USDT'
        self.correlation_timeframe = '1d'  # BTC trendini hangi zaman diliminde analiz edeceğiz
        self.correlation_period = 20  # Trend belirlemek için son kaç mum analiz edilecek
        
        self.excluded_coins = {
            'USDC', 'USDT', 'USTC', 'FDUSD', 'BUSD', 
            'TUSD', 'DAI', 'UST', 'USDD', 'USDP', 'SUSD'
        }

    def init_exchange_config(self):
        """Exchange yapılandırmasını başlatır"""
        self.exchange_config = {
            'apiKey': self.api_key,
            'secret': self.api_secret,
            'enableRateLimit': True,
            'timeout': 30000,
            'options': {
                'defaultType': 'spot',
                'adjustForTimeDifference': True,
                'recvWindow': 60000
            }
        }

    def create_exchange(self):
        """Her process için yeni bir exchange instance'ı oluşturur"""
        return ccxt.binance(self.exchange_config)

    def calculate_squeeze(self, ohlcv, period=20):
        """TTM Squeeze hesaplama"""
        if len(ohlcv) < period:
            return False
            
        close = ohlcv[:, 4]
        high = ohlcv[:, 2]
        low = ohlcv[:, 3]
        
        middle = np.mean(close[-period:])
        stddev = np.std(close[-period:])
        bb_upper = middle + (2 * stddev)
        bb_lower = middle - (2 * stddev)
        
        tr = np.maximum.reduce([
            high[-period:] - low[-period:],
            np.abs(high[-period:] - np.roll(close[-period:], 1)),
            np.abs(low[-period:] - np.roll(close[-period:], 1))
        ])
        atr = np.mean(tr)
        kc_upper = middle + (1.5 * atr)
        kc_lower = middle - (1.5 * atr)
        
        return (bb_lower > kc_lower) and (bb_upper < kc_upper)

    def calculate_fibonacci_gradient(self, df, timeframe):
        """Fibonacci gradient hesaplama"""
        lookback = self.lookback_periods[timeframe]
        gradients = np.zeros(len(df))
        
        for i in range(lookback, len(df)):
            window = df.iloc[i-lookback:i]
            high = window['high'].astype(float)
            low = window['low'].astype(float)
            close = float(df['close'].iloc[i])
            
            fib_max = high.max()
            fib_min = low.min()
            max_min_diff = fib_max - fib_min
            
            if max_min_diff <= 0:
                continue
                
            fib_sell = fib_max - (max_min_diff * 0.382)
            fib_buy = fib_max - (max_min_diff * 0.707)
            
            if close >= fib_sell:
                sell_diff = ((close - fib_sell) / max_min_diff * 100)
                gradient = ((sell_diff * 2617.801) + 100000) / 2
            elif close <= fib_buy:
                buy_diff = ((fib_buy - close) / max_min_diff * 100)
                gradient = ((buy_diff * -2617.801) + 100000) / 2
            else:
                gradient = 50000
                
            gradients[i] = (100 - gradient/1000) * -1
            
        return gradients

    def detect_support_resistance(self, df, n=5, threshold_pct=0.02, min_touches=2):
        """
        Destek ve direnç seviyelerini tespit eden fonksiyon
        
        Parametreler:
            df (pandas.DataFrame): OHLCV verisi içeren DataFrame
            n (int): Yerel minimum/maksimum tespiti için bakılacak nokta sayısı
            threshold_pct (float): Birbirine yakın seviyeleri birleştirmek için eşik değeri (%)
            min_touches (int): Bir seviyenin geçerli sayılması için min. test edilme sayısı
            
        Dönüş:
            dict: 'support' ve 'resistance' anahtarları ile destek ve direnç seviyelerini içeren sözlük
        """
        # Yerel maksimum ve minimum noktaları bul
        df_local_max = df.iloc[argrelextrema(df['high'].values, np.greater_equal, order=n)[0]]
        df_local_min = df.iloc[argrelextrema(df['low'].values, np.less_equal, order=n)[0]]
        
        # Seviye adaylarını oluştur (high ve low değerlerini kullan)
        resistance_levels = df_local_max['high'].values
        support_levels = df_local_min['low'].values
        
        # Birbirine çok yakın seviyeleri gruplayarak birleştir
        def merge_nearby_levels(levels, threshold_pct):
            if len(levels) == 0:
                return []
                
            # Seviyeleri sırala
            levels = sorted(levels)
            
            # Birleştirilmiş seviyeler
            merged_levels = []
            current_group = [levels[0]]
            
            for i in range(1, len(levels)):
                current_level = levels[i]
                prev_level = current_group[-1]
                
                # Eğer iki seviye arasındaki fark eşik değerinden küçükse, mevcut gruba ekle
                if abs(current_level - prev_level) / prev_level < threshold_pct:
                    current_group.append(current_level)
                else:
                    # Önceki grubu tamamla ve yeni bir grup başlat
                    merged_levels.append(sum(current_group) / len(current_group))
                    current_group = [current_level]
            
            # Son grubu ekle
            if current_group:
                merged_levels.append(sum(current_group) / len(current_group))
            
            return merged_levels
            
        merged_resistance = merge_nearby_levels(resistance_levels, threshold_pct)
        merged_support = merge_nearby_levels(support_levels, threshold_pct)
        
        # Seviyelerin kaç kez test edildiğini hesapla
        def count_touches(df, level, threshold_pct):
            """Belirli bir seviyenin kaç kez test edildiğini hesaplar"""
            threshold = level * threshold_pct
            touches = 0
            
            for _, row in df.iterrows():
                # Direnç seviyesi testi (yüksek fiyat seviyeye yakın)
                if abs(row['high'] - level) < threshold:
                    touches += 1
                # Destek seviyesi testi (düşük fiyat seviyeye yakın)
                elif abs(row['low'] - level) < threshold:
                    touches += 1
            
            return touches
        
        # Min. dokunma sayısını karşılayan seviyeleri filtrele
        valid_resistance = [level for level in merged_resistance 
                           if count_touches(df, level, threshold_pct) >= min_touches]
        valid_support = [level for level in merged_support 
                        if count_touches(df, level, threshold_pct) >= min_touches]
        
        return {
            'support': valid_support,
            'resistance': valid_resistance
        }
    
    def is_near_sr_level(self, price, levels, threshold_pct=0.02):
        """
        Fiyatın herhangi bir destek/direnç seviyesine yakın olup olmadığını kontrol eder
        
        Parametreler:
            price (float): Kontrol edilecek fiyat
            levels (list): Kontrol edilecek seviyeler listesi
            threshold_pct (float): Yakınlık eşik değeri (%)
            
        Dönüş:
            bool: Eğer fiyat herhangi bir seviyeye yakınsa True, değilse False
        """
        if not levels:
            return False
            
        for level in levels:
            if abs(price - level) / level < threshold_pct:
                return True
                
        return False
    
    async def get_market_leader_trend(self, exchange):
        """
        Piyasa lideri olan coinin (BTC) trendini analiz eder
        
        Parametreler:
            exchange: CCXT exchange nesnesi
            
        Dönüş:
            dict: Trend bilgilerini içeren sözlük
        """
        try:
            # Piyasa lideri için OHLCV verisi çek
            ohlcv = await exchange.fetch_ohlcv(
                self.market_leader, 
                self.correlation_timeframe, 
                limit=self.correlation_period
            )
            
            if not ohlcv or len(ohlcv) < self.correlation_period:
                print(f"Yetersiz veri: {self.market_leader}")
                return {'trend': 'neutral', 'strength': 0}
                
            # OHLCV verilerini DataFrame'e çevir
            df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
            
            # Kısa ve uzun dönem hareketli ortalamalar hesapla
            df['ma_short'] = df['close'].rolling(window=7).mean()
            df['ma_long'] = df['close'].rolling(window=20).mean()
            
            # Son değerleri al
            last_close = df['close'].iloc[-1]
            last_ma_short = df['ma_short'].iloc[-1]
            last_ma_long = df['ma_long'].iloc[-1]
            
            # RSI hesapla (14 periyod)
            delta = df['close'].diff()
            gain = delta.where(delta > 0, 0)
            loss = -delta.where(delta < 0, 0)
            
            avg_gain = gain.rolling(window=14).mean()
            avg_loss = loss.rolling(window=14).mean()
            
            rs = avg_gain / avg_loss
            rsi = 100 - (100 / (1 + rs))
            last_rsi = rsi.iloc[-1]
            
            # Son 7 günün trendinini hesapla (% değişim)
            price_change_pct = (df['close'].iloc[-1] - df['close'].iloc[-7]) / df['close'].iloc[-7] * 100
            
            # Trend belirle
            trend = 'neutral'
            strength = 0
            
            if last_ma_short > last_ma_long and last_close > last_ma_short:
                trend = 'bullish' 
                strength = 1
                if last_rsi > 60 and price_change_pct > 5:
                    strength = 2  # Güçlü yükseliş
            elif last_ma_short < last_ma_long and last_close < last_ma_short:
                trend = 'bearish'
                strength = -1
                if last_rsi < 40 and price_change_pct < -5:
                    strength = -2  # Güçlü düşüş
            
            return {
                'trend': trend,
                'strength': strength,
                'rsi': last_rsi,
                'price_change_pct': price_change_pct
            }
            
        except Exception as e:
            print(f"Piyasa lideri trend analizi hatası: {str(e)}")
            return {'trend': 'neutral', 'strength': 0}

    async def get_active_markets(self):
        """Aktif marketleri tarar ve filtreler"""
        exchange = self.create_exchange()
        try:
            print("\nAktif marketler taranıyor...")
            markets = await exchange.load_markets()
            active_pairs = []
            
            usdt_pairs = [s for s in markets if s.endswith('/USDT')]
            for symbol in tqdm(usdt_pairs, desc="Market Taraması", ncols=100):  # İlk 50 market ile test
                try:
                    ticker = await exchange.fetch_ticker(symbol)
                    if ticker['quoteVolume'] >= self.min_volume:
                        active_pairs.append(symbol)
                except Exception as e:
                    continue
                    
                await asyncio.sleep(0.5)  # Rate limit için bekleme süresi arttırıldı
            
            return active_pairs
        finally:
            await exchange.close()

    async def process_market(self, symbol):
        """Tek bir market için analiz yapar"""
        exchange = self.create_exchange()
        try:
            # Piyasa lideri trendini al
            market_trend = await self.get_market_leader_trend(exchange)
            
            results, ohlcv_data, extra_data = await self.analyze_market(exchange, symbol, market_trend)
            if results and symbol in results:
                return symbol, results[symbol], ohlcv_data, extra_data
            return None
        finally:
            await exchange.close()

    async def analyze_market(self, exchange, symbol, market_trend):
        """Market analizi yapan fonksiyon - destek/direnç ve piyasa korelasyonu eklendi"""
        results = {}
        ohlcv_data = {}
        timeframe_data = {}
        extra_data = {'support_resistance': {}, 'market_trend': market_trend}
        
        for timeframe in self.timeframes:
            try:
                ohlcv = await exchange.fetch_ohlcv(
                    symbol, 
                    timeframe, 
                    limit=self.timeframe_limits[timeframe]
                )
                
                if not ohlcv or len(ohlcv) < self.lookback_periods[timeframe]:
                    print(f"Yetersiz veri: {symbol} - {timeframe}")
                    return results, ohlcv_data, extra_data
                    
                timeframe_data[timeframe] = {
                    'ohlcv': ohlcv,
                    'array': np.array(ohlcv),
                    'df': pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
                }
                
                # Destek ve direnç seviyelerini hesapla
                df = timeframe_data[timeframe]['df']
                sr_levels = self.detect_support_resistance(
                    df.tail(self.sr_lookback), 
                    n=self.sr_order,
                    threshold_pct=self.sr_threshold,
                    min_touches=self.sr_touch_count
                )
                extra_data['support_resistance'][timeframe] = sr_levels
                
            except Exception as e:
                print(f"Veri çekme hatası ({symbol} - {timeframe}): {str(e)}")
                return results, ohlcv_data, extra_data
                
            await asyncio.sleep(0.5)

        all_signals = {}
        for timeframe, data in timeframe_data.items():
            gradients = self.calculate_fibonacci_gradient(data['df'], timeframe)
            last_gradient = gradients[-1]
            
            if last_gradient <= -70.7:
                signal = "Alış"
            elif last_gradient >= -38.2:
                signal = "Satış"
            else:
                return results, ohlcv_data, extra_data
                
            # Piyasa korelasyonu kontrolü - BTC düşüş trendindeyse alış sinyallerinin güvenilirliğini düşür
            signal_confidence = 1.0  # Başlangıç güven değeri
            
            if signal == "Alış" and market_trend['trend'] == 'bearish':
                # BTC düşüş trendindeyse alış sinyallerinin güvenilirliğini azalt
                signal_confidence -= 0.3 * abs(market_trend['strength'])
            elif signal == "Satış" and market_trend['trend'] == 'bullish':
                # BTC yükseliş trendindeyse satış sinyallerinin güvenilirliğini azalt
                signal_confidence -= 0.3 * abs(market_trend['strength'])
                
            # Destek/Direnç kontrolü
            current_price = data['df']['close'].iloc[-1]
            sr_levels = extra_data['support_resistance'][timeframe]
            
            # Fiyat destek seviyesine yakınsa ve sinyal Alış ise güvenilirliği artır
            if signal == "Alış" and self.is_near_sr_level(current_price, sr_levels['support']):
                signal_confidence += 0.2
            
            # Fiyat direnç seviyesine yakınsa ve sinyal Satış ise güvenilirliği artır
            if signal == "Satış" and self.is_near_sr_level(current_price, sr_levels['resistance']):
                signal_confidence += 0.2
                
            all_signals[timeframe] = {
                'signal': signal,
                'gradient': last_gradient,
                'confidence': min(1.0, max(0.0, signal_confidence))  # 0-1 arasında sınırla
            }
        
        first_signal = list(all_signals.values())[0]['signal']
        # Tüm zaman dilimlerinde aynı sinyal olması gerekiyor
        if not all(signal['signal'] == first_signal for signal in all_signals.values()):
            return results, ohlcv_data, extra_data
        
        # Eğer tüm sinyaller çok düşük güven skoruna sahipse filtrele
        avg_confidence = sum(signal['confidence'] for signal in all_signals.values()) / len(all_signals)
        if avg_confidence < 0.5:  # Ortalama güven 0.5'ten düşükse sinyali reddet
            return results, ohlcv_data, extra_data
            
        has_squeeze = False
        for timeframe, data in timeframe_data.items():
            if self.calculate_squeeze(data['array']):
                has_squeeze = True
                break
        
        if has_squeeze:
            results[symbol] = all_signals
            ohlcv_data = {tf: data['ohlcv'] for tf, data in timeframe_data.items()}
            
        return results, ohlcv_data, extra_data

    def plot_analysis(self, symbol, data, ohlcv_data, extra_data):
        """Grafik oluşturma fonksiyonu - Destek/Direnç ve market korelasyonu eklendi"""
        fig = make_subplots(
            rows=2, 
            cols=2, 
            subplot_titles=[f"{symbol} - {self.timeframe_names[tf]}" for tf in self.timeframes],
            vertical_spacing=0.1,
            horizontal_spacing=0.05
        )

        colors = {
            '15m': 'rgba(255, 0, 0, 0.3)', 
            '1h': 'rgba(0, 255, 0, 0.3)', 
            '4h': 'rgba(0, 0, 255, 0.3)',
            '1d': 'rgba(255, 165, 0, 0.3)'
        }

        # Piyasa trendi bilgisini al
        market_trend = extra_data['market_trend']
        trend_text = f"BTC/USDT Trend: {market_trend['trend'].upper()} (Güç: {market_trend['strength']})"
        
        for idx, timeframe in enumerate(self.timeframes):
            row = idx // 2 + 1
            col = idx % 2 + 1
            
            if timeframe in ohlcv_data:
                ohlcv = ohlcv_data[timeframe]
                df = pd.DataFrame(ohlcv, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
                df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
                
                gradients = self.calculate_fibonacci_gradient(df, timeframe)
                df['gradient'] = gradients
                
                fig.add_trace(
                    go.Candlestick(
                        x=df['timestamp'],
                        open=df['open'],
                        high=df['high'],
                        low=df['low'],
                        close=df['close'],
                        name=timeframe,
                        showlegend=False,
                        customdata=df['gradient'],
                        hovertext=[
                            f'Tarih: {date}<br>' +
                            f'Açılış: {open:.8f}<br>' +
                            f'Yüksek: {high:.8f}<br>' +
                            f'Düşük: {low:.8f}<br>' +
                            f'Kapanış: {close:.8f}<br>' +
                            f'Gradient: {gradient:.2f}'
                            for date, open, high, low, close, gradient in zip(
                                df['timestamp'], df['open'], df['high'], 
                                df['low'], df['close'], df['gradient']
                            )
                        ],
                        hoverinfo='text'
                    ),
                    row=row, col=col
                )

                # Destek ve direnç seviyelerini çiz
                if timeframe in extra_data['support_resistance']:
                    sr_levels = extra_data['support_resistance'][timeframe]
                    
                    # Destek seviyeleri için yatay çizgiler
                    for level in sr_levels['support']:
                        fig.add_shape(
                            type="line",
                            x0=df['timestamp'].iloc[0],
                            x1=df['timestamp'].iloc[-1],
                            y0=level,
                            y1=level,
                            line=dict(color="green", width=1, dash="dash"),
                            row=row, col=col
                        )
                    
                    # Direnç seviyeleri için yatay çizgiler
                    for level in sr_levels['resistance']:
                        fig.add_shape(
                            type="line",
                            x0=df['timestamp'].iloc[0],
                            x1=df['timestamp'].iloc[-1],
                            y0=level,
                            y1=level,
                            line=dict(color="red", width=1, dash="dash"),
                            row=row, col=col
                        )

                for i in range(1, len(df)):
                    is_squeeze = self.calculate_squeeze(np.array(ohlcv[max(0, i-20):i+1]))
                    if is_squeeze:
                        fig.add_shape(
                            type="rect",
                            x0=df['timestamp'].iloc[i-1],
                            x1=df['timestamp'].iloc[i],
                            y0=df['low'].min(),
                            y1=df['high'].max(),
                            fillcolor=colors[timeframe],
                            opacity=0.3,
                            layer="below",
                            line_width=0,
                            row=row,
                            col=col
                        )

                if timeframe in data.get(symbol, {}):
                    signal_info = data[symbol][timeframe]
                    confidence = signal_info.get('confidence', 1.0)
                    confidence_text = f" (Güven: {confidence:.2f})"
                    
                    if signal_info['signal'] == "Alış":
                        fig.add_trace(
                            go.Scatter(
                                x=[df['timestamp'].iloc[-1]],
                                y=[df['low'].iloc[-1] * 0.999],
                                mode='markers',
                                marker=dict(
                                    symbol='triangle-up', 
                                    size=12, 
                                    color='lime',
                                    opacity=confidence  # Güven değerine göre opaklık ayarla
                                ),
                                name=f'Alım Sinyali{confidence_text}',
                                showlegend=idx == 0
                            ),
                            row=row, col=col
                        )
                    elif signal_info['signal'] == "Satış":
                        fig.add_trace(
                            go.Scatter(
                                x=[df['timestamp'].iloc[-1]],
                                y=[df['high'].iloc[-1] * 1.001],
                                mode='markers',
                                marker=dict(
                                    symbol='triangle-down', 
                                    size=12, 
                                    color='red',
                                    opacity=confidence  # Güven değerine göre opaklık ayarla
                                ),
                                name=f'Satım Sinyali{confidence_text}',
                                showlegend=idx == 0
                            ),
                            row=row, col=col
                        )

                fig.update_xaxes(
                    title_text="Tarih",
                    gridcolor='rgba(128,128,128,0.2)',
                    rangeslider=dict(visible=False),
                    row=row, 
                    col=col,
                    tickfont=dict(size=10)
                )
                fig.update_yaxes(
                    title_text="Fiyat",
                    gridcolor='rgba(128,128,128,0.2)',
                    row=row, 
                    col=col,
                    tickformat='.8f',
                    tickfont=dict(size=10)
                )

        # Genel grafik stili
        fig.update_layout(
            height=1200,
            width=1800,
            template="plotly_dark",
            paper_bgcolor='black',
            plot_bgcolor='black',
            title=dict(                 
                font=dict(size=28, color='white'),
                x=0.5,
                y=0.95
            ),
            showlegend=True,
            legend=dict(
                yanchor="top",
                y=0.99,
                xanchor="right",
                x=0.99,
                bgcolor='rgba(0,0,0,0.5)',
                font=dict(size=12, color='white')
            ),
            margin=dict(t=80, b=50, l=50, r=50)
        )

        # Piyasa trendi bilgisini renklendirerek belirt
        trend_color = "green" if market_trend['trend'] == 'bullish' else "red" if market_trend['trend'] == 'bearish' else "white"
        
        # Trend bilgisini açıklama olarak ekle
        fig.add_annotation(
            xref="paper", yref="paper",
            x=0.5, y=1.05,
            text=trend_text,
            showarrow=False,
            font=dict(size=14, color=trend_color)
        )

        # Subplot başlıkları için renk ayarı
        for annotation in fig['layout']['annotations']:
            annotation['font'] = dict(size=14, color='white')

        return fig

    async def process_chunk(self, chunk):
        """Bir grup marketi işler"""
        tasks = []
        for symbol in chunk:
            tasks.append(self.process_market(symbol))
            await asyncio.sleep(0.1)  # Rate limit kontrolü
        return await asyncio.gather(*tasks, return_exceptions=True)

    async def run_analysis(self):
        """Ana analiz fonksiyonu"""
        try:
            markets = await self.get_active_markets()
            if not markets:
                print("Aktif market bulunamadı!")
                return

            print(f"\nToplam {len(markets)} aktif market bulundu")
            
            # Daha küçük chunk'lar kullan
            chunk_size = 5  # Chunk boyutu küçültüldü
            chunks = [markets[i:i + chunk_size] for i in range(0, len(markets), chunk_size)]
            
            results = []
            for chunk in tqdm(chunks, desc="Analiz İlerlemesi", ncols=100):
                try:
                    chunk_results = await self.process_chunk(chunk)
                    valid_results = [r for r in chunk_results if r is not None and not isinstance(r, Exception)]
                    results.extend(valid_results)
                    await asyncio.sleep(1)  # Chunklar arası bekleme
                except Exception as e:
                    print(f"Chunk işleme hatası: {str(e)}")
                    continue

            print("\n=== ÖZET ===")
            print(f"Toplam {len(results)} coin için sinyal bulundu:")
            
            for symbol, signals, ohlcv_data, extra_data in results:
                try:
                    print(f"\n{symbol} Sinyal Özeti:")
                    for tf, result in signals.items():
                        print(f"✓ {self.timeframe_names[tf]}: {result['signal']} ({result['gradient']:.2f})")
                    
                    fig = self.plot_analysis(symbol, {symbol: signals}, ohlcv_data, extra_data)
                    filename = f"{symbol.replace('/', '_')}_{datetime.now().strftime('%Y%m%d_%H%M')}.png"
                    fig.write_image(filename, engine="kaleido", scale=2)
                    
                    # Belleği temizle
                    del fig
                except Exception as e:
                    print(f"{symbol} için grafik oluşturma hatası: {str(e)}")
                    continue

        except Exception as e:
            print(f"Genel hata: {str(e)}")
            import traceback
            traceback.print_exc()
   
    @classmethod
    async def create_and_run(cls):
        """Sınıfın instance'ını oluşturup çalıştıran factory metod"""
        scanner = cls()
        await scanner.run_analysis()
        return scanner

def run_scanner():
    """Jupyter için çalıştırma fonksiyonu"""
    try:
        scanner = TradingScanner()
        asyncio.run(scanner.run_analysis())
    except Exception as e:
        print(f"\nBir hata oluştu: {e}")
        import traceback
        traceback.print_exc()

if __name__ == "__main__":
    run_scanner()
